<h3>모델 생성</h3>

라이브러리 호출

In [69]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import joblib
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import mean_absolute_error, r2_score

데이터 로드 및 전처리

In [45]:
ship_data = pd.read_excel('ship_data.xlsx')
truck_data = pd.read_excel('truck_data.xlsx')

In [46]:
ship_data['입항일시'] = pd.to_datetime(ship_data['입항일시'])
ship_data['출항일시'] = pd.to_datetime(ship_data['출항일시'])
truck_data['입문시각'] = pd.to_datetime(truck_data['입문시각'])
truck_data['출문시각'] = pd.to_datetime(truck_data['출문시각'])

사용할 열

In [47]:
ship_data = ship_data[['입항일시', '출항일시', '계선장소', 'number']]
truck_data = truck_data[['입문시각', '출문시각','차종']]

배의 입항시간과 출항시간 사이에 있는 화물차 데이터 병합 및 선박 개수 계산

In [48]:
def merge_data_with_ship_count(ship_data, truck_data):
    # 병합된 데이터를 저장할 리스트 초기화
    merged_data = []
    # 배 데이터의 각 행을 반복 처리
    for _, ship_row in ship_data.iterrows():
        # 현재 배의 입항시간과 출항시간 사이에 있는 화물차 데이터를 필터링
        relevant_trucks = truck_data[
            # 화물차의 입문시각이 배의 입항시간 이후
            (truck_data['입문시각'] >= ship_row['입항일시']) &
            # 화물차의 출문시각이 배의 출항시간 이전
            (truck_data['출문시각'] <= ship_row['출항일시']) 
        ]

        # 해당 배의 기간 내에 있는 화물차 수를 계산
        ship_count = relevant_trucks.shape[0]

        # 필터링된 각 화물차 데이터에 대해 반복 처리
        for _, truck_row in relevant_trucks.iterrows():
            # 화물차 데이터를 필요한 형식으로 리스트에 추가
            merged_data.append([
                truck_row['입문시각'].year,
                truck_row['입문시각'].month,
                truck_row['입문시각'].day,
                truck_row['입문시각'].hour,
                truck_row['차종'],
                ship_count,
                (truck_row['출문시각'] - truck_row['입문시각']).total_seconds() / 60
                # 입문시각과 출문시각 사이의 시간(분 단위)
            ])

        # 병합된 데이터를 데이터 프레임으로 변환하여 반환
        # return 문의 위치 주의하기(안쪽에 있으면 truck만 반환함)
        return pd.DataFrame(merged_data, columns=[
                '입문시각_연도','입문시각_월', '입문시각_일', '입문시각_시간',
                '차종', '선박_갯수', '걸린시간'
        ])

데이터 병합 및 확인

In [49]:
merged_data = merge_data_with_ship_count(ship_data, truck_data)
print(merged_data.head(10))

   입문시각_연도  입문시각_월  입문시각_일  입문시각_시간           차종  선박_갯수  걸린시간
0     2022       5      19       13  화물 소형(1t미만)    575   8.0
1     2022       5      19       10  화물 대형(5t이상)    575  22.0
2     2022       5      19        7  화물 소형(1t미만)    575  50.0
3     2022       5      19       13  화물 대형(5t이상)    575  26.0
4     2022       5      19       10  화물 대형(5t이상)    575   6.0
5     2022       5      19       16  화물 대형(5t이상)    575  41.0
6     2022       5      19        9  화물 소형(1t미만)    575  88.0
7     2022       5      20        3  화물 대형(5t이상)    575  16.0
8     2022       5      19        8  화물 대형(5t이상)    575  34.0
9     2022       5      19       13  화물 대형(5t이상)    575  62.0


데이터 문자열로 변환

In [50]:
merged_data['차종'] = merged_data['차종'].astype(str)

데이터 분리

In [51]:
# 특성 데이터
X = merged_data.drop(['걸린시간'], axis=1)
# 타겟 데이터
y = merged_data['걸린시간'] 

레이블 인코딩

In [52]:
label_encoders = {}
for column in X.select_dtypes(include=['object']).columns:
    label_encoders[column] = LabelEncoder()
    X[column] = label_encoders[column].fit_transform(X[column])

데이터 분할

In [53]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

텐서로 변환

In [54]:
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)

데이터셋 및 데이터로더 생성

In [55]:
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(train_dataset, batch_size=32, shuffle=False)


LSTM 모델 정의

In [56]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        # LSTM 계층 정의
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        # 완전결의층 정의
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # 초기 hidden state와 cell state를 0으로 초기화
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        # LSTM 계층을 통과한 출력값과 상태값
        out, _ = self.lstm(x, (h0, c0))
        # 마지막 시점의 출력을 완전연결층에 통과시켜 최종 출력값을 계산
        out = self.fc(out[:, -1, :])
        return out

모델 인스턴스 생성

In [57]:
# 입력 특성의 크기
input_size = X_train.shape[1]
# LSTM의 은닉상태 크기
hidden_size = 50
# LSTM 계층의 수
num_layers = 2
# 출력 크기 (예측하려는 타겟 변수의 수)
output_size = 1
model = LSTMModel(input_size, hidden_size, num_layers, output_size)

손실 함수와 옵티마이저 정의

In [58]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

모델 학습

In [59]:
num_epochs = 50
model.train()
for epoch in range(num_epochs):
    for features, targets in train_loader:
        # LSTM 입력 형태로 변환(batch_size,seq_length, input_size)
        features = features.unsqueeze(1)
        # 타겟 텐서의 차원 추가
        targets = targets.unsqueeze(1)
        # 모델 예측
        outputs = model(features)
        # 손실 계산
        loss = criterion(outputs, targets)

        # 기울기 초기화
        optimizer.zero_grad()
        # 역전파 수행
        loss.backward()
        # 가중치 업데이트
        optimizer.step()
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss : {loss.item():.4f}')
    # f는 포맷 문자열 리터럴 의미(f-string)
    # 현재 에포크 번호와 총 에포크 수를 표시
    # 손실 값을 소수점 네번째 자리까지 표시
    # 각 에포크의 손실 값은 모델이 예측한 값과 실제 값 사이의 평균제곱 오차(MSE)를 나타냄

Epoch [1/50], Loss : 2845.2141
Epoch [2/50], Loss : 6321.8716
Epoch [3/50], Loss : 1729.1299
Epoch [4/50], Loss : 2144.0950
Epoch [5/50], Loss : 4753.6602
Epoch [6/50], Loss : 4578.6733
Epoch [7/50], Loss : 2201.7058
Epoch [8/50], Loss : 2694.1968
Epoch [9/50], Loss : 2980.1758
Epoch [10/50], Loss : 2437.4084
Epoch [11/50], Loss : 657.0085
Epoch [12/50], Loss : 2355.1406
Epoch [13/50], Loss : 2995.6299
Epoch [14/50], Loss : 1460.4067
Epoch [15/50], Loss : 2477.7522
Epoch [16/50], Loss : 1113.1063
Epoch [17/50], Loss : 2791.5186
Epoch [18/50], Loss : 1852.4003
Epoch [19/50], Loss : 1232.2968
Epoch [20/50], Loss : 608.6743
Epoch [21/50], Loss : 1164.9745
Epoch [22/50], Loss : 2178.6843
Epoch [23/50], Loss : 1654.8541
Epoch [24/50], Loss : 1311.6403
Epoch [25/50], Loss : 4524.7803
Epoch [26/50], Loss : 2401.0046
Epoch [27/50], Loss : 1678.2565
Epoch [28/50], Loss : 2530.4778
Epoch [29/50], Loss : 878.6728
Epoch [30/50], Loss : 1864.7969
Epoch [31/50], Loss : 833.1910
Epoch [32/50], Loss :

모델 평가

In [60]:
model.eval()
with torch.no_grad():
    test_loss = 0
    for features, targets in test_loader:
        # LSTM 입력 형태로 변환(batch_size,seq_length, input_size)
        features = features.unsqueeze(1)
        # 타겟 텐서의 차원 추가
        targets = targets.unsqueeze(1)
        # 모델 예측
        outputs = model(features)
        # 손실 계산
        loss = criterion(outputs, targets)
        # 테스트 손실 누적
        test_loss += loss.item()

    # 평균 테스트 손실 계산
    test_loss /= len(test_loader)
    print(f'Test Loss : {test_loss:.4f}')

    # 1번째 시도 ) Test Loss : 1267.3691
    # 2번째 시도 ) Test Loss : 1271.1031

Test Loss : 1271.1031


훈련 성능(train score), 테스트 성능(test score), 평균 절대 오차(mean absolute error, MAE) 계산

In [70]:
# 훈련 데이터 예측
model.train()
with torch.no_grad():
    train_outputs = model(X_train_tensor.unsqueeze(1))
    train_predictions = train_outputs.squeeze(1).numpy()
    train_mae = mean_absolute_error(y_train, train_predictions)
    train_r2 = r2_score(y_train, train_predictions)
    print(f"Train MAE: {train_mae:.4f}, Train R2: {train_r2:.4f}")

Train MAE: 25.6906, Train R2: -0.1150


In [71]:
# 테스트 데이터 예측
model.eval()
with torch.no_grad():
    test_outputs = model(X_test_tensor.unsqueeze(1))
    test_predictions = test_outputs.squeeze(1).numpy()
    test_mae = mean_absolute_error(y_test, test_predictions)
    test_r2 = r2_score(y_test, test_predictions)
    print(f"Test MAE: {test_mae:.4f}, Test R2: {test_r2:.4f}")

Test MAE: 28.4074, Test R2: -0.1691


모델 전체 저장

In [62]:
torch.save(model, 'lstm_model.pth')

<h3>예측 수행</h3>

모델 불러오기

In [64]:
# 저장된 모델 불러오기
model = torch.load('lstm_model.pth')

predict_processing_time 함수 정의

In [66]:
def predict_processing_time(model, year, month, day, hour, truck_type, ship_count):
    # 입력 데이터를 데이터프레임으로 생성
    input_data = pd.DataFrame([{
        '입문시각_연도': year,
        '입문시각_월': month,
        '입문시각_일': day,
        '입문시각_시간': hour,
        '차종': truck_type,
        '선박_갯수': ship_count
    }])

    # 문자열 컬럼을 레이블 인코딩(숫자로 변환)
    for column in input_data.select_dtypes(include=['object']).columns:
        # 사전에 정의된 레이블 인코더가 있는 경우
        if column in label_encoders:
            input_data[column] = label_encoders[column].transform(input_data[column])

    # 데이터를 텐서로 변환
    input_tensor = torch.tensor(input_data.values, dtype=torch.float32)
    # LSTM 입력 형태로 변환
    input_tensor = input_tensor.unsqueeze(1)

    # 모델을 평가 모드로 전환
    model.eval()
    # 그래디언트 계산 비활성화(메모리 절약)
    with torch.no_grad():
        # 모델 예측 수행
        predicted_time = model(input_tensor)
    
    # 예측 결과를 숫자로 반환
    return predicted_time.item()


차량 크기 함수 정의

In [67]:
def truck_size(truck_type):
    if truck_type in ["컨테이너차량", "특수차량(중장비)", "화물 대형(5t이상)"]:
        return "대"
    elif truck_type in ["화물 중형(1t이상~5t미만)"]:
        return "중"
    elif  truck_type in ["화물 소형(1t미만)"]:
        return "소"
    else:
        return "알수없음"

예시로 하나의 예측 수행

In [68]:
predicted_time = predict_processing_time(model, 2024, 7, 1, 1, '화물 대형(5t이상)', 1)

print(f"예상 입출문 소요시간: 약 {predicted_time} 분, 차량 크기: {truck_size('화물 대형(5t이상)')}")

예상 입출문 소요시간: 약 28.88765525817871 분, 차량 크기: 대
